<a href="https://colab.research.google.com/github/ChrisHallak/LLM-Fine-Tunning/blob/main/continue_checkpoint_fine_tunning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/gdrive")

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
!pip install -qU datasets==3.2.0 optimum==1.24.0
!pip install -qU openai==1.61.0 wandb
!pip install -qU json-repair==0.29.1
!pip install PyPDF2
!pip install transformers -U
!pip install bitsandbytes -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.6/433.6 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import userdata
import wandb

huggingface_key = userdata.get('huggingface')
wandb_key = userdata.get('wandb')
openai_key = userdata.get('openai')

wandb.login(key=wandb_key)
!huggingface-cli login --token {huggingface_key}


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: christeenhallak33 (christeenhallak33-coretech-mena) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `hf`CLI if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `huggingfaceToken` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `huggingfaceToken`


In [ ]:
import json
import os
from os.path import join
import random
from tqdm.auto import tqdm
import requests

from pydantic import BaseModel, Field
from typing import List, Optional, Literal
from datetime import datetime

import json_repair

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

data_dir = '/gdrive/MyDrive/Artificial Intelligence/LLM/Fine Tunning/question_dataset_1000.jsonl'
saved_dir = '/gdrive/MyDrive/Artificial Intelligence/LLM/Fine Tunning'
gbase_model_id = "Qwen/Qwen2.5-1.5B-Instruct"

device = "cuda"
torch_dtype = None

def parse_json(text):
    try:
        return json_repair.loads(text)
    except:
        return None

In [ ]:
!git clone "https://github.com/hiyouga/LLaMA-Factory.git"

Cloning into 'LLaMA-Factory'...
remote: Enumerating objects: 24550, done.
remote: Counting objects: 100% (146/146), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 24550 (delta 118), reused 54 (delta 53), pack-reused 24404 (from 3)
Receiving objects: 100% (24550/24550), 53.33 MiB | 13.26 MiB/s, done.
Resolving deltas: 100% (17709/17709), done.


In [ ]:
# # Configure LLaMA-Factory for the new datasets

# # update /content/LLaMA-Factory/data/dataset_info.json and append
'''
        "news_finetune_train": {
            "file_name": "/gdrive/MyDrive/Artificial Intelligence/LLM/Fine Tunning/datasets/llamafactory-finetune-data/train.json",
            "columns": {
                "prompt": "instruction",
                "query": "input",
                "response": "output",
                "system": "system",
                "history": "history"
            }
        },
        "news_finetune_val": {
            "file_name": "/gdrive/MyDrive/Artificial Intelligence/LLM/Fine Tunning/datasets/llamafactory-finetune-data/val.json",
            "columns": {
                "prompt": "instruction",
                "query": "input",
                "response": "output",
                "system": "system",
                "history": "history"
            }
        }
'''

# https://wandb.ai/mr-bakrianoo/llamafactory/runs/apwbkni9
# https://wandb.ai/mr-bakrianoo/llamafactory/runs/c5tf0q90

'\n        "news_finetune_train": {\n            "file_name": "/gdrive/MyDrive/Artificial Intelligence/LLM/Fine Tunning/datasets/llamafactory-finetune-data/train.json",\n            "columns": {\n                "prompt": "instruction",\n                "query": "input",\n                "response": "output",\n                "system": "system",\n                "history": "history"\n            }\n        },\n        "news_finetune_val": {\n            "file_name": "/gdrive/MyDrive/Artificial Intelligence/LLM/Fine Tunning/datasets/llamafactory-finetune-data/val.json",\n            "columns": {\n                "prompt": "instruction",\n                "query": "input",\n                "response": "output",\n                "system": "system",\n                "history": "history"\n            }\n        }\n'

In [ ]:
%%writefile /content/LLaMA-Factory/examples/train_lora/news_finetune.yaml

### model
model_name_or_path: Qwen/Qwen2.5-1.5B-Instruct
trust_remote_code: true

### method
stage: sft
do_train: true
finetuning_type: lora
lora_rank: 64
lora_target: all

### dataset
dataset: news_finetune_train
eval_dataset: news_finetune_val
template: qwen
cutoff_len: 3500
# max_samples: 50
overwrite_cache: true
preprocessing_num_workers: 16

### output
resume_from_checkpoint: /gdrive/MyDrive/Artificial Intelligence/LLM/Fine Tunning/Qwen_models/checkpoint-550
output_dir: /gdrive/MyDrive/Artificial Intelligence/LLM/Fine Tunning/Qwen_models/
logging_steps: 10
save_strategy: steps
save_steps: 50
plot_loss: true
overwrite_output_dir: true

### train
per_device_train_batch_size: 1
gradient_accumulation_steps: 4
learning_rate: 1.0e-4
num_train_epochs: 3.0
lr_scheduler_type: cosine
warmup_ratio: 0.1
bf16: true
ddp_timeout: 180000000

### eval
# val_size: 0.1
per_device_eval_batch_size: 1
eval_strategy: steps
eval_steps: 50

report_to: wandb
run_name: questions_generations_Qwen

push_to_hub: true
export_hub_model_id: "Christeen33/questions_generations_Qwen"
hub_private_repo: true
hub_strategy: checkpoint


Overwriting /content/LLaMA-Factory/examples/train_lora/news_finetune.yaml


In [ ]:
# # STEP 1: Clone the repo
# !git clone https://github.com/hiyouga/LLaMA-Factory.git

# STEP 2: Change directory
%cd LLaMA-Factory

# STEP 3: Install in editable mode
!pip install -e .

# Force restart the runtime (only works in Colab)
import os
os.kill(os.getpid(), 9)  # This will crash the runtime and force a restart

# After restarting, run only the training command in a new cell:
# !llamafactory-cli train examples/train_lora/news_finetune.yaml

/content/LLaMA-Factory
Obtaining file:///content/LLaMA-Factory
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.1/362.1 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 114.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 110.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.1/411.1 kB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.

In [ ]:
%cd /content/LLaMA-Factory


/content/LLaMA-Factory


In [ ]:

# STEP 4: Run training using the example config
!llamafactory-cli train examples/train_lora/news_finetune.yaml


2025-08-10 07:07:45.091879: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754809665.354789    4404 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754809665.429266    4404 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1754809665.985699    4404 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754809665.985743    4404 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754809665.985747    4404 computation_placer.cc:177] computation placer alr